# 1. Importing necessary libraries 

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
pd.set_option("display.max_columns",500)

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import optuna

# 2. Importing the dataset

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e18/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e18/test.csv')

In [3]:
train

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,ExactMolWt,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,HallKierAlpha,HeavyAtomMolWt,Kappa3,MaxAbsEStateIndex,MinEStateIndex,NumHeteroatoms,PEOE_VSA10,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
0,0,323.390782,9.879918,5.875576,5.875576,4.304757,4.304757,2.754513,1.749203,0.000000,11.938294,222.068080,1.181818,1.727273,2.363636,-0.24,212.163,8.170000,11.922504,0.171585,4,0.000000,91.536492,0.000000,0.000000,0.000000,17.744066,0.000000,4.794537,35.527357,0,0,1,1,0,0,0,0
1,1,273.723798,7.259037,4.441467,5.834958,3.285046,4.485235,2.201375,1.289775,45.135471,0.000000,260.029719,1.346154,2.076923,2.769231,-0.09,247.031,3.201491,10.932338,-4.830450,10,24.415866,7.822697,0.000000,0.000000,0.000000,7.822697,30.705892,13.825658,44.707310,0,0,0,1,1,0,0,0
2,2,521.643822,10.911303,8.527859,11.050864,6.665291,9.519706,5.824822,1.770579,15.645394,6.606882,382.131027,1.085714,1.742857,2.400000,-0.78,354.106,15.033890,11.238048,-5.066255,9,0.000000,15.645394,0.000000,53.378235,0.000000,15.645394,73.143616,17.964475,45.660120,0,0,1,1,0,0,1,0
3,3,567.431166,12.453343,7.089119,12.833709,6.478023,10.978151,7.914542,3.067181,95.639554,0.000000,530.070277,1.162791,1.573770,2.270270,-1.30,506.124,6.724301,11.171170,-5.276575,19,42.727765,21.335138,0.000000,0.000000,6.420822,15.645394,62.107304,31.961948,87.509997,0,0,1,1,0,0,0,0
4,4,112.770735,4.414719,2.866236,2.866236,1.875634,1.875634,1.036450,0.727664,17.980451,12.841643,118.062994,1.444444,2.111111,2.555556,-1.10,108.056,3.931272,9.855741,-1.676296,4,6.041841,11.938611,6.923737,19.386400,0.000000,11.938611,18.883484,9.589074,33.333333,2,2,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14833,14833,632.207041,10.911303,6.579933,9.179964,4.653583,6.030052,3.670528,1.770579,32.971529,6.606882,347.063084,1.562500,2.187500,2.687500,-1.87,333.112,4.879408,11.426427,-5.077909,16,17.248535,15.645394,0.000000,13.847474,6.923737,34.407699,32.607024,18.947452,61.376610,0,0,1,1,0,0,0,0
14834,14834,62.568425,2.642734,1.446898,1.446898,0.879497,0.879497,0.174620,0.000000,0.000000,0.000000,74.024203,2.666667,2.666667,2.666667,-0.53,70.027,0.729375,5.506944,0.250000,0,0.000000,0.000000,0.000000,0.000000,6.066367,0.000000,6.420822,0.000000,10.000000,0,0,0,1,0,1,0,0
14835,14835,981.327476,10.363081,6.146219,6.146219,4.700576,4.700576,3.064846,2.133897,17.248535,0.000000,297.089560,1.461538,2.153846,2.769231,-3.15,282.220,2.045502,12.118907,-0.446105,8,34.480943,0.000000,0.000000,0.000000,23.762553,10.969244,0.000000,0.000000,66.666667,0,0,1,1,0,0,0,0
14836,14836,299.171248,9.949161,6.589761,7.848913,5.276568,5.476436,3.978973,2.299833,45.623794,0.000000,265.959270,0.950000,1.400000,1.950000,-0.44,257.119,8.424399,10.909292,-4.181527,7,0.000000,7.822697,19.420579,0.000000,0.000000,7.822697,108.961047,9.088795,45.583333,0,0,0,1,1,0,0,0


In [4]:
train.isna().sum()

id                   0
BertzCT              0
Chi1                 0
Chi1n                0
Chi1v                0
Chi2n                0
Chi2v                0
Chi3v                0
Chi4n                0
EState_VSA1          0
EState_VSA2          0
ExactMolWt           0
FpDensityMorgan1     0
FpDensityMorgan2     0
FpDensityMorgan3     0
HallKierAlpha        0
HeavyAtomMolWt       0
Kappa3               0
MaxAbsEStateIndex    0
MinEStateIndex       0
NumHeteroatoms       0
PEOE_VSA10           0
PEOE_VSA14           0
PEOE_VSA6            0
PEOE_VSA7            0
PEOE_VSA8            0
SMR_VSA10            0
SMR_VSA5             0
SlogP_VSA3           0
VSA_EState9          0
fr_COO               0
fr_COO2              0
EC1                  0
EC2                  0
EC3                  0
EC4                  0
EC5                  0
EC6                  0
dtype: int64

In [5]:
train.dtypes

id                     int64
BertzCT              float64
Chi1                 float64
Chi1n                float64
Chi1v                float64
Chi2n                float64
Chi2v                float64
Chi3v                float64
Chi4n                float64
EState_VSA1          float64
EState_VSA2          float64
ExactMolWt           float64
FpDensityMorgan1     float64
FpDensityMorgan2     float64
FpDensityMorgan3     float64
HallKierAlpha        float64
HeavyAtomMolWt       float64
Kappa3               float64
MaxAbsEStateIndex    float64
MinEStateIndex       float64
NumHeteroatoms         int64
PEOE_VSA10           float64
PEOE_VSA14           float64
PEOE_VSA6            float64
PEOE_VSA7            float64
PEOE_VSA8            float64
SMR_VSA10            float64
SMR_VSA5             float64
SlogP_VSA3           float64
VSA_EState9          float64
fr_COO                 int64
fr_COO2                int64
EC1                    int64
EC2                    int64
EC3           

In [6]:
train.describe()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,ExactMolWt,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,HallKierAlpha,HeavyAtomMolWt,Kappa3,MaxAbsEStateIndex,MinEStateIndex,NumHeteroatoms,PEOE_VSA10,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
count,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000
mean,7418.500000,515.153604,9.135189,5.854307,6.738497,4.432570,5.253221,3.418749,1.773472,29.202823,10.435316,292.623087,1.236774,1.812070,2.255470,-1.207776,274.950211,5.874372,10.556443,-2.119772,8.584108,11.021644,17.790011,8.962440,11.318811,6.704487,15.666766,31.066423,13.636941,49.309959,0.458215,0.459226,0.667745,0.798962,0.313789,0.279081,0.144831,0.151570
std,4283.505982,542.456370,6.819989,4.647064,5.866444,3.760516,4.925065,3.436208,1.865898,31.728679,13.651843,225.384140,5.491284,5.495565,5.501200,0.935314,212.678755,45.730226,1.559331,2.066415,7.643769,13.958962,34.561655,19.756727,20.169745,10.865415,18.080208,33.896638,14.598554,29.174824,0.667948,0.668111,0.471038,0.400790,0.464047,0.448562,0.351942,0.358616
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007276,-666.000000,-666.000000,-666.000000,-7.730000,0.000000,-104.040000,0.000000,-6.327514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.430556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3709.250000,149.103601,4.680739,2.844556,2.932842,1.949719,2.034468,1.160763,0.503897,5.969305,0.000000,148.037173,1.045455,1.690909,2.100000,-1.660000,136.109000,1.784008,9.926190,-4.659604,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.969305,6.420822,4.794537,30.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,7418.500000,290.987941,6.485270,4.052701,4.392859,2.970427,3.242775,1.948613,1.073261,17.353601,6.420822,206.042653,1.250000,1.865152,2.358491,-1.100000,194.276500,3.261011,10.421334,-1.265370,6.000000,6.041841,5.969305,0.000000,0.000000,0.000000,11.752550,20.075376,9.589074,41.666667,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,11127.750000,652.652585,11.170477,7.486791,8.527859,5.788793,6.609350,4.502070,2.534281,44.876559,12.841643,343.090331,1.500000,2.062153,2.500000,-0.570000,326.002000,5.848400,11.539743,-0.787037,10.000000,18.311899,15.645394,12.132734,13.847474,6.923737,17.721856,42.727765,14.912664,56.090650,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,14837.000000,4069.959780,69.551167,50.174588,53.431954,32.195368,34.579313,22.880836,16.072810,363.705954,99.936429,2237.318490,3.000000,3.200000,3.400000,0.820000,2035.133000,1512.242231,15.630251,6.000000,42.000000,97.663462,482.434223,375.425148,211.501279,100.348416,80.742293,492.729739,115.406157,384.450519,8.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
cols = train.drop(['id','EC1','EC2','EC3','EC4','EC5','EC6'],axis=1).columns

# 3. Scaling the data

In [8]:
ms = MinMaxScaler()
train[cols] = ms.fit_transform(train[cols])

In [9]:
train.describe()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,ExactMolWt,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,HallKierAlpha,HeavyAtomMolWt,Kappa3,MaxAbsEStateIndex,MinEStateIndex,NumHeteroatoms,PEOE_VSA10,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
count,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000
mean,7418.500000,0.126575,0.131345,0.116679,0.126114,0.137677,0.151918,0.149415,0.110340,0.080292,0.104420,0.130400,0.997364,0.997926,0.998290,0.762833,0.135102,0.068004,0.675385,0.341329,0.204384,0.112853,0.036876,0.023873,0.053517,0.066812,0.194034,0.063050,0.118165,0.140403,0.057277,0.057403,0.667745,0.798962,0.313789,0.279081,0.144831,0.151570
std,4283.505982,0.133283,0.098057,0.092618,0.109793,0.116803,0.142428,0.150178,0.116090,0.087237,0.136605,0.100784,0.008208,0.008212,0.008218,0.109393,0.104504,0.028293,0.099764,0.167626,0.181995,0.142929,0.071640,0.052625,0.095365,0.108277,0.223925,0.068794,0.126497,0.074830,0.083493,0.083514,0.471038,0.400790,0.464047,0.448562,0.351942,0.358616
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3709.250000,0.036635,0.067299,0.056693,0.054889,0.060559,0.058835,0.050731,0.031351,0.016412,0.000000,0.065747,0.997078,0.997745,0.998058,0.709942,0.066880,0.065474,0.635063,0.135300,0.095238,0.000000,0.000000,0.000000,0.000000,0.000000,0.073930,0.013031,0.041545,0.090875,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,7418.500000,0.071497,0.093245,0.080772,0.082214,0.092263,0.093778,0.085164,0.066775,0.047713,0.064249,0.091685,0.997384,0.998005,0.998444,0.775439,0.095461,0.066388,0.666741,0.410638,0.142857,0.061864,0.012373,0.000000,0.000000,0.000000,0.145556,0.040743,0.083090,0.120799,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,11127.750000,0.160358,0.160608,0.149215,0.159602,0.179802,0.191136,0.196762,0.157675,0.123387,0.128498,0.152968,0.997758,0.998300,0.998656,0.837427,0.160187,0.067988,0.738295,0.449440,0.238095,0.187500,0.032430,0.032317,0.065472,0.068997,0.219487,0.086716,0.129219,0.157795,0.125000,0.125000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,14837.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# 4. Data preparation

In [10]:
X = train[cols]
Y = train[['EC1','EC2']]

In [11]:
x_train_full, x, y_train_full, y = train_test_split(X,Y,test_size=0.3,random_state=123)
x_val, x_test, y_val, y_test = train_test_split(x, y, test_size=0.5,random_state=123)
x_train, x_, y_train, y_ = train_test_split(x_train_full,y_train_full,test_size=0.2,random_state=123)
len(x_train), len(x_val) , len(x_test), len(x_)

(8308, 2226, 2226, 2078)

# 5. LGBM Classifier Model | Optuna Hyperparamter Search

In [12]:
def objective_lgbm_ec1(trial):
    n_leaves = trial.suggest_int('num_leaves', 25,100)
    max_depth = trial.suggest_int("max_depth", 1,10)
    n_estimators = trial.suggest_int("n_estimators", 10,2000)
    r_alpha = trial.suggest_int("reg_alpha", 0.0, 0.1)
    r_lambda = trial.suggest_int("reg_lambda", 0.0, 0.1)
    l_rate = trial.suggest_float('learning_rate', 0.001, 0.1,log=True)
    subsample =  trial.suggest_float('subsample', 0.5, 1.0)
    lambda_l1 = trial.suggest_int("lambda_l1", 0.0, 4)
    lambda_l2 = trial.suggest_int("lambda_l2", 0.0, 4)
    feature_fraction =  trial.suggest_float('feature_fraction', 0.5, 1.0)

    lgb = LGBMClassifier(
            num_leaves =n_leaves,
            max_depth=max_depth, 
            n_estimators=n_estimators,
            reg_alpha = r_alpha,
            reg_lambda = r_lambda,
            learning_rate = l_rate,
            subsample = subsample,
            lambda_l1 = lambda_l1,
            lambda_l2 = lambda_l2,
            feature_fraction = feature_fraction
        )
    
    lgb.fit(x_train, y_train['EC1'], eval_set=[(x_val, y_val['EC1'])], early_stopping_rounds=20, verbose=False)

    y_pred_proba = lgb.predict_proba(x_val)[:, 1]
    roc_auc = roc_auc_score(y_val['EC1'], y_pred_proba)
    return roc_auc
study_lgbm_ec1 = optuna.create_study(direction="maximize")
study_lgbm_ec1.optimize(objective_lgbm_ec1, n_trials=30)

[I 2023-06-28 11:46:54,281] A new study created in memory with name: no-name-c806ebac-7fb9-4e9f-adcf-bccf70977b07
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] feature_fraction is set=0.588806434233037, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.588806434233037
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0 will be ignored. Current value: lambda_l2=3


[I 2023-06-28 11:46:55,488] Trial 0 finished with value: 0.6912311485440736 and parameters: {'num_leaves': 28, 'max_depth': 10, 'n_estimators': 1806, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.02309293913216091, 'subsample': 0.7999734518094546, 'lambda_l1': 0, 'lambda_l2': 3, 'feature_fraction': 0.588806434233037}. Best is trial 0 with value: 0.6912311485440736.
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argume

[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.5253044568008665, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5253044568008665
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0 will be ignored. Current value: lambda_l2=0


[I 2023-06-28 11:46:58,129] Trial 1 finished with value: 0.6882994996940576 and parameters: {'num_leaves': 46, 'max_depth': 4, 'n_estimators': 630, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.0020328763224852333, 'subsample': 0.8640651125924323, 'lambda_l1': 3, 'lambda_l2': 0, 'feature_fraction': 0.5253044568008665}. Best is trial 0 with value: 0.6912311485440736.
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argum

...

[I 2023-06-28 11:47:33,452] Trial 27 finished with value: 0.6844270777093907 and parameters: {'num_leaves': 74, 'max_depth': 7, 'n_estimators': 1490, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.06841592023837441, 'subsample': 0.6886170510866909, 'lambda_l1': 1, 'lambda_l2': 2, 'feature_fraction': 0.9944184359984819}. Best is trial 3 with value: 0.6939765324119066.
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argum

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] feature_fraction is set=0.7473950005925488, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7473950005925488
[LightGBM] [Warning] lambda_l2 is set=4, reg_lambda=0 will be ignored. Current value: lambda_l2=4


[I 2023-06-28 11:47:34,651] Trial 28 finished with value: 0.6907461397257315 and parameters: {'num_leaves': 47, 'max_depth': 8, 'n_estimators': 244, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.028133470670867643, 'subsample': 0.6476479297784278, 'lambda_l1': 0, 'lambda_l2': 4, 'feature_fraction': 0.7473950005925488}. Best is trial 3 with value: 0.6939765324119066.
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argum

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] feature_fraction is set=0.5447783836868838, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5447783836868838
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0 will be ignored. Current value: lambda_l2=3


[I 2023-06-28 11:47:35,253] Trial 29 finished with value: 0.6914790519382356 and parameters: {'num_leaves': 41, 'max_depth': 5, 'n_estimators': 516, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.037278700389625545, 'subsample': 0.7727486582905739, 'lambda_l1': 0, 'lambda_l2': 3, 'feature_fraction': 0.5447783836868838}. Best is trial 3 with value: 0.6939765324119066.


In [13]:
trial_lgbm_ec1 = study_lgbm_ec1.best_trial
print('ROC_AUC Score: {}'.format(trial_lgbm_ec1.value))
print("Best hyperparameters: {}".format(trial_lgbm_ec1.params))

ROC_AUC Score: 0.6939765324119066
Best hyperparameters: {'num_leaves': 33, 'max_depth': 9, 'n_estimators': 290, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.09383073807047086, 'subsample': 0.5149475034484587, 'lambda_l1': 0, 'lambda_l2': 2, 'feature_fraction': 0.8377231358054495}


In [14]:
def objective_lgbm_ec2(trial):
    n_leaves = trial.suggest_int('num_leaves', 25,100)
    max_depth = trial.suggest_int("max_depth", 1,10)
    n_estimators = trial.suggest_int("n_estimators", 10,2000)
    r_alpha = trial.suggest_int("reg_alpha", 0.0, 0.1)
    r_lambda = trial.suggest_int("reg_lambda", 0.0, 0.1)
    l_rate = trial.suggest_float('learning_rate', 0.001, 0.1,log=True)
    subsample =  trial.suggest_float('subsample', 0.5, 1.0)
    lambda_l1 = trial.suggest_int("lambda_l1", 0.0, 4)
    lambda_l2 = trial.suggest_int("lambda_l2", 0.0, 4)
    feature_fraction =  trial.suggest_float('feature_fraction', 0.5, 1.0)

    lgb = LGBMClassifier(
            num_leaves =n_leaves,
            max_depth=max_depth, 
            n_estimators=n_estimators,
            reg_alpha = r_alpha,
            reg_lambda = r_lambda,
            learning_rate = l_rate,
            subsample = subsample,
            lambda_l1 = lambda_l1,
            lambda_l2 = lambda_l2,
            feature_fraction = feature_fraction
        )
    
    lgb.fit(x_train, y_train['EC2'], eval_set=[(x_val, y_val['EC2'])], early_stopping_rounds=20, verbose=False)

    y_pred_proba = lgb.predict_proba(x_val)[:, 1]
    roc_auc = roc_auc_score(y_val['EC2'], y_pred_proba)
    return roc_auc
study_lgbm_ec2 = optuna.create_study(direction="maximize")
study_lgbm_ec2.optimize(objective_lgbm_ec2, n_trials=30)

[I 2023-06-28 11:47:35,392] A new study created in memory with name: no-name-61fe91b1-e46e-481e-9c29-2a0acb91c3ec
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=4, reg_alpha=0 will be ignored. Current value: lambda_l1=4
[LightGBM] [Warning] feature_fraction is set=0.5404283825835957, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5404283825835957
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0 will be ignored. Current value: lambda_l2=2


[I 2023-06-28 11:47:38,934] Trial 0 finished with value: 0.5790717740716481 and parameters: {'num_leaves': 75, 'max_depth': 5, 'n_estimators': 1459, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.0034116512365711645, 'subsample': 0.8197955091944968, 'lambda_l1': 4, 'lambda_l2': 2, 'feature_fraction': 0.5404283825835957}. Best is trial 0 with value: 0.5790717740716481.
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argu

[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.5812895075166464, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5812895075166464
[LightGBM] [Warning] lambda_l2 is set=4, reg_lambda=0 will be ignored. Current value: lambda_l2=4


[I 2023-06-28 11:47:43,131] Trial 1 finished with value: 0.5742983826270973 and parameters: {'num_leaves': 54, 'max_depth': 9, 'n_estimators': 1101, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.003628839887302343, 'subsample': 0.7345584388976334, 'lambda_l1': 3, 'lambda_l2': 4, 'feature_fraction': 0.5812895075166464}. Best is trial 0 with value: 0.5790717740716481.
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argum

...

[I 2023-06-28 11:48:22,552] Trial 27 finished with value: 0.5792745755025949 and parameters: {'num_leaves': 60, 'max_depth': 1, 'n_estimators': 820, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.002560677920276023, 'subsample': 0.6959483043850871, 'lambda_l1': 1, 'lambda_l2': 3, 'feature_fraction': 0.7040618579797124}. Best is trial 21 with value: 0.5894026805058699.
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argu

[LightGBM] [Warning] lambda_l1 is set=2, reg_alpha=0 will be ignored. Current value: lambda_l1=2
[LightGBM] [Warning] feature_fraction is set=0.8306089544207785, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8306089544207785
[LightGBM] [Warning] lambda_l2 is set=4, reg_lambda=0 will be ignored. Current value: lambda_l2=4


[I 2023-06-28 11:48:24,464] Trial 28 finished with value: 0.5775129742530358 and parameters: {'num_leaves': 32, 'max_depth': 2, 'n_estimators': 1054, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.0013394530882244763, 'subsample': 0.5902423581754187, 'lambda_l1': 2, 'lambda_l2': 4, 'feature_fraction': 0.8306089544207785}. Best is trial 21 with value: 0.5894026805058699.
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' ar

[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.889901592456873, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.889901592456873
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0 will be ignored. Current value: lambda_l2=2


[I 2023-06-28 11:48:28,189] Trial 29 finished with value: 0.5730708671335719 and parameters: {'num_leaves': 71, 'max_depth': 6, 'n_estimators': 1609, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.003186672804508503, 'subsample': 0.5250590191388823, 'lambda_l1': 1, 'lambda_l2': 2, 'feature_fraction': 0.889901592456873}. Best is trial 21 with value: 0.5894026805058699.


In [15]:
trial_lgbm_ec2 = study_lgbm_ec2.best_trial
print('ROC_AUC Score: {}'.format(trial_lgbm_ec2.value))
print("Best hyperparameters: {}".format(trial_lgbm_ec1.params))

ROC_AUC Score: 0.5894026805058699
Best hyperparameters: {'num_leaves': 33, 'max_depth': 9, 'n_estimators': 290, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.09383073807047086, 'subsample': 0.5149475034484587, 'lambda_l1': 0, 'lambda_l2': 2, 'feature_fraction': 0.8377231358054495}


In [16]:
lgbm_ec1 = LGBMClassifier(**trial_lgbm_ec1.params)
lgbm_ec1.fit(x_train, y_train['EC1'], eval_set=[(x_val, y_val['EC1'])], early_stopping_rounds=20, verbose=False)

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] feature_fraction is set=0.8377231358054495, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8377231358054495
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0 will be ignored. Current value: lambda_l2=2


LGBMClassifier(feature_fraction=0.8377231358054495, lambda_l1=0, lambda_l2=2,
               learning_rate=0.09383073807047086, max_depth=9, n_estimators=290,
               num_leaves=33, reg_alpha=0, reg_lambda=0,
               subsample=0.5149475034484587)

In [17]:
y_pred_proba = lgbm_ec1.predict_proba(x_test)[:, 1]
roc_auc_score(y_test['EC1'], y_pred_proba)

0.6948145557172694

In [18]:
lgbm_ec2 = LGBMClassifier(**trial_lgbm_ec2.params)
lgbm_ec2.fit(x_train, y_train['EC2'], eval_set=[(x_val, y_val['EC2'])], early_stopping_rounds=20, verbose=False)

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.7713094644346348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7713094644346348
[LightGBM] [Warning] lambda_l2 is set=4, reg_lambda=0 will be ignored. Current value: lambda_l2=4


LGBMClassifier(feature_fraction=0.7713094644346348, lambda_l1=1, lambda_l2=4,
               learning_rate=0.0011403507662051577, max_depth=1,
               n_estimators=806, num_leaves=25, reg_alpha=0, reg_lambda=0,
               subsample=0.6308589121580126)

In [19]:
y_pred_proba = lgbm_ec2.predict_proba(x_test)[:, 1]
roc_auc_score(y_test['EC2'], y_pred_proba)

0.601062861975296

# 6. CatBoost Classifier Model | Optuna Hyperparamter Search

In [20]:
def objective_cb_ec1(trial):
    iterations = trial.suggest_int('iterations', 1000,5000)
    max_depth = trial.suggest_int("max_depth", 1,10)
    l_rate = trial.suggest_float('learning_rate', 0.001, 0.1,log=True)
    subsample =  trial.suggest_float('subsample', 0.5, 1.0)
    l2_leaf_reg = trial.suggest_int("l2_leaf_reg", 0.0, 4)
    colsample_bylevel =  trial.suggest_float('colsample_bylevel', 0.5, 1.0)

    cb = CatBoostClassifier(
            iterations =iterations,
            max_depth=max_depth, 
            learning_rate = l_rate,
            subsample = subsample,
            l2_leaf_reg = l2_leaf_reg,
            colsample_bylevel = colsample_bylevel
        )
    
    cb.fit(x_train, y_train['EC1'], eval_set=[(x_val, y_val['EC1'])], early_stopping_rounds=20, verbose=100)

    y_pred_proba = cb.predict_proba(x_val)[:, 1]
    roc_auc = roc_auc_score(y_val['EC1'], y_pred_proba)
    return roc_auc
study_cb_ec1 = optuna.create_study(direction="maximize")
study_cb_ec1.optimize(objective_cb_ec1, n_trials=30)

[I 2023-06-28 11:48:30,145] A new study created in memory with name: no-name-2ec29af9-8eb0-410b-bed2-e53f97a1d9e8


0:	learn: 0.6920683	test: 0.6921508	best: 0.6921508 (0)	total: 64.3ms	remaining: 2m 8s
100:	learn: 0.6216616	test: 0.6290983	best: 0.6290983 (100)	total: 830ms	remaining: 15.6s
200:	learn: 0.5903189	test: 0.6044643	best: 0.6044643 (200)	total: 1.59s	remaining: 14.2s
300:	learn: 0.5737984	test: 0.5943386	best: 0.5943386 (300)	total: 2.35s	remaining: 13.2s
400:	learn: 0.5633968	test: 0.5903161	best: 0.5903161 (400)	total: 3.1s	remaining: 12.4s
500:	learn: 0.5555964	test: 0.5890139	best: 0.5889634 (498)	total: 3.85s	remaining: 11.5s


[I 2023-06-28 11:48:34,881] Trial 0 finished with value: 0.6908838138429975 and parameters: {'iterations': 1998, 'max_depth': 6, 'learning_rate': 0.004578759979437687, 'subsample': 0.986401760200892, 'l2_leaf_reg': 0, 'colsample_bylevel': 0.8821879335405867}. Best is trial 0 with value: 0.6908838138429975.


Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.5884334613
bestIteration = 566

Shrink model to first 567 iterations.
0:	learn: 0.6924555	test: 0.6924822	best: 0.6924822 (0)	total: 7.18ms	remaining: 7.62s
100:	learn: 0.6415361	test: 0.6457064	best: 0.6457064 (100)	total: 736ms	remaining: 7s
200:	learn: 0.6125982	test: 0.6200867	best: 0.6200867 (200)	total: 1.46s	remaining: 6.24s
300:	learn: 0.5953613	test: 0.6057525	best: 0.6057525 (300)	total: 2.21s	remaining: 5.59s
400:	learn: 0.5845755	test: 0.5980489	best: 0.5980489 (400)	total: 2.94s	remaining: 4.84s
500:	learn: 0.5771692	test: 0.5935551	best: 0.5935551 (500)	total: 3.67s	remaining: 4.1s
600:	learn: 0.5719406	test: 0.5909347	best: 0.5909347 (600)	total: 4.39s	remaining: 3.36s
700:	learn: 0.5679607	test: 0.5895421	best: 0.5895421 (700)	total: 5.27s	remaining: 2.71s
800:	learn: 0.5645415	test: 0.5887397	best: 0.5887397 (800)	total: 6.08s	remaining: 1.97s
900:	learn: 0.5616329	test: 0.5884163	best: 0.5884003 (897)

[I 2023-06-28 11:48:42,629] Trial 1 finished with value: 0.6902647302307165 and parameters: {'iterations': 1061, 'max_depth': 6, 'learning_rate': 0.002974377156486166, 'subsample': 0.6165076508533966, 'l2_leaf_reg': 1, 'colsample_bylevel': 0.9328558360115304}. Best is trial 0 with value: 0.6908838138429975.


...

100:	learn: 0.5620447	test: 0.5894854	best: 0.5893712 (92)	total: 486ms	remaining: 19s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.5893711739
bestIteration = 92

Shrink model to first 93 iterations.
0:	learn: 0.6769582	test: 0.6784284	best: 0.6784284 (0)	total: 32.9ms	remaining: 2m 28s


[I 2023-06-28 11:51:50,399] Trial 28 finished with value: 0.693512219702696 and parameters: {'iterations': 4519, 'max_depth': 9, 'learning_rate': 0.07237905435337169, 'subsample': 0.5915495955915966, 'l2_leaf_reg': 2, 'colsample_bylevel': 0.680577546919944}. Best is trial 17 with value: 0.6946158622179031.


Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.5875194916
bestIteration = 34

Shrink model to first 35 iterations.
0:	learn: 0.6787620	test: 0.6794210	best: 0.6794210 (0)	total: 11.4ms	remaining: 51.8s


[I 2023-06-28 11:51:51,496] Trial 29 finished with value: 0.6918119893460029 and parameters: {'iterations': 4531, 'max_depth': 7, 'learning_rate': 0.06825298814411629, 'subsample': 0.5920701179196562, 'l2_leaf_reg': 2, 'colsample_bylevel': 0.692643576990237}. Best is trial 17 with value: 0.6946158622179031.


Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.5880323992
bestIteration = 54

Shrink model to first 55 iterations.


In [21]:
trial_cb_ec1 = study_cb_ec1.best_trial
print('ROC_AUC Score: {}'.format(trial_cb_ec1.value))
print("Best hyperparameters: {}".format(trial_cb_ec1.params))

ROC_AUC Score: 0.6946158622179031
Best hyperparameters: {'iterations': 3358, 'max_depth': 10, 'learning_rate': 0.05245000563306037, 'subsample': 0.5694730982549754, 'l2_leaf_reg': 4, 'colsample_bylevel': 0.6511533476707224}


In [22]:
def objective_cb_ec2(trial):
    iterations = trial.suggest_int('iterations', 1000,5000)
    max_depth = trial.suggest_int("max_depth", 1,10)
    l_rate = trial.suggest_float('learning_rate', 0.001, 0.1,log=True)
    subsample =  trial.suggest_float('subsample', 0.5, 1.0)
    l2_leaf_reg = trial.suggest_int("l2_leaf_reg", 0.0, 4)
    colsample_bylevel =  trial.suggest_float('colsample_bylevel', 0.5, 1.0)

    cb = CatBoostClassifier(
            iterations = iterations,
            max_depth=max_depth, 
            learning_rate = l_rate,
            subsample = subsample,
            l2_leaf_reg = l2_leaf_reg,
            colsample_bylevel = colsample_bylevel
        )
    
    cb.fit(x_train, y_train['EC2'], eval_set=[(x_val, y_val['EC2'])], early_stopping_rounds=20, verbose=100)

    y_pred_proba = cb.predict_proba(x_val)[:, 1]
    roc_auc = roc_auc_score(y_val['EC2'], y_pred_proba)
    return roc_auc
study_cb_ec2 = optuna.create_study(direction="maximize")
study_cb_ec2.optimize(objective_cb_ec2, n_trials=30)

[I 2023-06-28 11:51:51,784] A new study created in memory with name: no-name-2ef585f6-21a4-4c25-8cd2-801ce8bea9a1


0:	learn: 0.6641163	test: 0.6638140	best: 0.6638140 (0)	total: 2.89ms	remaining: 13.4s


[I 2023-06-28 11:51:52,296] Trial 0 finished with value: 0.5810103542097043 and parameters: {'iterations': 4648, 'max_depth': 1, 'learning_rate': 0.07492650799733089, 'subsample': 0.8895186900831222, 'l2_leaf_reg': 1, 'colsample_bylevel': 0.9634452854503879}. Best is trial 0 with value: 0.5810103542097043.


100:	learn: 0.4975146	test: 0.4949140	best: 0.4947522 (90)	total: 290ms	remaining: 13.1s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.494603017
bestIteration = 118

Shrink model to first 119 iterations.
0:	learn: 0.6633978	test: 0.6633002	best: 0.6633002 (0)	total: 8.57ms	remaining: 23s


[I 2023-06-28 11:51:52,830] Trial 1 finished with value: 0.5771363430241346 and parameters: {'iterations': 2688, 'max_depth': 6, 'learning_rate': 0.07530826804967497, 'subsample': 0.5045381625066584, 'l2_leaf_reg': 2, 'colsample_bylevel': 0.8746288080083596}. Best is trial 0 with value: 0.5810103542097043.


...

0:	learn: 0.6880927	test: 0.6880662	best: 0.6880662 (0)	total: 4.95ms	remaining: 17.6s
100:	learn: 0.5126454	test: 0.5116388	best: 0.5116388 (100)	total: 478ms	remaining: 16.4s
200:	learn: 0.4964418	test: 0.4961463	best: 0.4961463 (200)	total: 916ms	remaining: 15.3s
300:	learn: 0.4918169	test: 0.4939957	best: 0.4939822 (295)	total: 1.38s	remaining: 14.9s


[I 2023-06-28 11:53:36,711] Trial 28 finished with value: 0.5888805612938983 and parameters: {'iterations': 3553, 'max_depth': 4, 'learning_rate': 0.012465859301069459, 'subsample': 0.7627730088649908, 'l2_leaf_reg': 4, 'colsample_bylevel': 0.7817705813859382}. Best is trial 14 with value: 0.5894946339497154.


400:	learn: 0.4883068	test: 0.4935899	best: 0.4935378 (396)	total: 1.86s	remaining: 14.6s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.4935378127
bestIteration = 396

Shrink model to first 397 iterations.
0:	learn: 0.6766124	test: 0.6764172	best: 0.6764172 (0)	total: 2.75ms	remaining: 11.7s
100:	learn: 0.4994571	test: 0.4956228	best: 0.4956228 (100)	total: 245ms	remaining: 10s


[I 2023-06-28 11:53:37,358] Trial 29 finished with value: 0.5856212525822543 and parameters: {'iterations': 4233, 'max_depth': 1, 'learning_rate': 0.04191148411379291, 'subsample': 0.8450901123373453, 'l2_leaf_reg': 0, 'colsample_bylevel': 0.9213442985571303}. Best is trial 14 with value: 0.5894946339497154.


200:	learn: 0.4968090	test: 0.4941572	best: 0.4941199 (180)	total: 496ms	remaining: 9.96s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.4941198838
bestIteration = 180

Shrink model to first 181 iterations.


In [23]:
trial_cb_ec2 = study_cb_ec2.best_trial
print('ROC_AUC Score: {}'.format(trial_cb_ec2.value))
print("Best hyperparameters: {}".format(trial_cb_ec2.params))

ROC_AUC Score: 0.5894946339497154
Best hyperparameters: {'iterations': 3559, 'max_depth': 4, 'learning_rate': 0.0087208607148801, 'subsample': 0.699372937195524, 'l2_leaf_reg': 3, 'colsample_bylevel': 0.8012374433343337}


In [24]:
cb_ec1 = CatBoostClassifier(**trial_cb_ec1.params)
cb_ec1.fit(x_train, y_train['EC1'], eval_set=[(x_val, y_val['EC1'])], early_stopping_rounds=20, verbose=False)

In [25]:
y_pred_proba = cb_ec1.predict_proba(x_test)[:, 1]
roc_auc_score(y_test['EC1'], y_pred_proba)

0.6994613915773678

In [26]:
cb_ec2 = CatBoostClassifier(**trial_cb_ec2.params)
cb_ec2.fit(x_train, y_train['EC2'], eval_set=[(x_val, y_val['EC2'])], early_stopping_rounds=20, verbose=False)

In [27]:
y_pred_proba = cb_ec2.predict_proba(x_test)[:, 1]
roc_auc_score(y_test['EC2'], y_pred_proba)

0.6167008354261698

# 7. XGBoost Classifier Model | Optuna Hyper Parameter Search

In [28]:
def objective_xgb_ec1(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 2500)
    max_depth = trial.suggest_int("max_depth", 1,10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1,log=True)
    subsample =  trial.suggest_float('subsample', 0.5, 1.0)
    r_alpha = trial.suggest_int("reg_alpha", 0.0, 0.1)
    r_lambda = trial.suggest_int("reg_lambda", 0.0, 0.1)
    colsample_bylevel =  trial.suggest_float('colsample_bylevel', 0.5, 1.0)

    xgb = XGBClassifier(
            n_estimators =n_estimators,
            max_depth=max_depth, 
            learning_rate = learning_rate,
            subsample = subsample,
            reg_alpha = r_alpha,
            reg_lambda = r_lambda,
            colsample_bylevel = colsample_bylevel
        )
    
    xgb.fit(x_train, y_train['EC1'], eval_set=[(x_val, y_val['EC1'])], early_stopping_rounds=20, verbose=100)

    y_pred_proba = xgb.predict_proba(x_val)[:, 1]
    roc_auc = roc_auc_score(y_val['EC1'], y_pred_proba)
    return roc_auc
study_xgb_ec1 = optuna.create_study(direction="maximize")
study_xgb_ec1.optimize(objective_xgb_ec1, n_trials=30)

[I 2023-06-28 11:53:43,071] A new study created in memory with name: no-name-c9e3daa1-8088-49b8-9caf-dae193dd0a1f


[0]	validation_0-logloss:0.69167


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-logloss:0.61091
[154]	validation_0-logloss:0.59796


[I 2023-06-28 11:53:50,773] Trial 0 finished with value: 0.6888740416801642 and parameters: {'n_estimators': 155, 'max_depth': 10, 'learning_rate': 0.007444767033813142, 'subsample': 0.7326899320716066, 'reg_alpha': 0, 'reg_lambda': 0, 'colsample_bylevel': 0.7468168942178863}. Best is trial 0 with value: 0.6888740416801642.


...

[I 2023-06-28 12:05:30,528] Trial 28 finished with value: 0.690607115862218 and parameters: {'n_estimators': 2126, 'max_depth': 4, 'learning_rate': 0.0022852758247360887, 'subsample': 0.8944450908421433, 'reg_alpha': 0, 'reg_lambda': 0, 'colsample_bylevel': 0.6204525777041169}. Best is trial 12 with value: 0.6925305042651981.


[0]	validation_0-logloss:0.69226


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-logloss:0.63305
[200]	validation_0-logloss:0.60720
[300]	validation_0-logloss:0.59551
[400]	validation_0-logloss:0.59063
[484]	validation_0-logloss:0.58883


[I 2023-06-28 12:05:40,802] Trial 29 finished with value: 0.692352337760501 and parameters: {'n_estimators': 485, 'max_depth': 6, 'learning_rate': 0.0044043674444351226, 'subsample': 0.747335844425485, 'reg_alpha': 0, 'reg_lambda': 0, 'colsample_bylevel': 0.5327080922066113}. Best is trial 12 with value: 0.6925305042651981.


In [29]:
trial_xgb_ec1 = study_xgb_ec1.best_trial
print('ROC_AUC Score: {}'.format(trial_xgb_ec1.value))
print("Best hyperparameters: {}".format(trial_xgb_ec1.params))

ROC_AUC Score: 0.6925305042651981
Best hyperparameters: {'n_estimators': 2500, 'max_depth': 6, 'learning_rate': 0.0010052496318172302, 'subsample': 0.8242348594808815, 'reg_alpha': 0, 'reg_lambda': 0, 'colsample_bylevel': 0.5045401102524364}


In [30]:
xgb_ec1 = XGBClassifier(**trial_xgb_ec1.params)
xgb_ec1.fit(x_train, y_train['EC1'], eval_set=[(x_val, y_val['EC1'])], early_stopping_rounds=20, verbose=False)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.5045401102524364, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0010052496318172302,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=2500, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [31]:
def objective_xgb_ec2(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 2500)
    max_depth = trial.suggest_int("max_depth", 1,10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1,log=True)
    subsample =  trial.suggest_float('subsample', 0.5, 1.0)
    r_alpha = trial.suggest_int("reg_alpha", 0.0, 0.1)
    r_lambda = trial.suggest_int("reg_lambda", 0.0, 0.1)
    colsample_bylevel =  trial.suggest_float('colsample_bylevel', 0.5, 1.0)

    xgb = XGBClassifier(
            n_estimators =n_estimators,
            max_depth=max_depth, 
            learning_rate = learning_rate,
            subsample = subsample,
            reg_alpha = r_alpha,
            reg_lambda = r_lambda,
            colsample_bylevel = colsample_bylevel
        )
    
    xgb.fit(x_train, y_train['EC2'], eval_set=[(x_val, y_val['EC2'])], early_stopping_rounds=20, verbose=100)

    y_pred_proba = xgb.predict_proba(x_val)[:, 1]
    roc_auc = roc_auc_score(y_val['EC2'], y_pred_proba)
    return roc_auc
study_xgb_ec2 = optuna.create_study(direction="maximize")
study_xgb_ec2.optimize(objective_xgb_ec2, n_trials=30)

[I 2023-06-28 12:06:30,872] A new study created in memory with name: no-name-d238ab5e-fd4c-4569-9ede-160034496cbf


[0]	validation_0-logloss:0.68848
[100]	validation_0-logloss:0.51477
[200]	validation_0-logloss:0.49676
[300]	validation_0-logloss:0.49543
[314]	validation_0-logloss:0.49548


[I 2023-06-28 12:06:34,940] Trial 0 finished with value: 0.5829268907139618 and parameters: {'n_estimators': 1647, 'max_depth': 3, 'learning_rate': 0.012861411224157899, 'subsample': 0.8859496138079741, 'reg_alpha': 0, 'reg_lambda': 0, 'colsample_bylevel': 0.7466866144267571}. Best is trial 0 with value: 0.5829268907139618.


[0]	validation_0-logloss:0.67677


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

[I 2023-06-28 12:08:54,564] Trial 28 finished with value: 0.5841481080264019 and parameters: {'n_estimators': 2084, 'max_depth': 3, 'learning_rate': 0.032041747010068344, 'subsample': 0.7082894819633031, 'reg_alpha': 0, 'reg_lambda': 0, 'colsample_bylevel': 0.7673382170570685}. Best is trial 15 with value: 0.5925454728674359.


[0]	validation_0-logloss:0.68833


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-logloss:0.51422
[200]	validation_0-logloss:0.49760
[300]	validation_0-logloss:0.49543
[400]	validation_0-logloss:0.49489
[468]	validation_0-logloss:0.49464


[I 2023-06-28 12:08:57,866] Trial 29 finished with value: 0.5834446011991737 and parameters: {'n_estimators': 1515, 'max_depth': 1, 'learning_rate': 0.013391201994018666, 'subsample': 0.766481492105883, 'reg_alpha': 0, 'reg_lambda': 0, 'colsample_bylevel': 0.7448262231576901}. Best is trial 15 with value: 0.5925454728674359.


In [32]:
trial_xgb_ec2 = study_xgb_ec2.best_trial
print('ROC_AUC Score: {}'.format(trial_xgb_ec2.value))
print("Best hyperparameters: {}".format(trial_xgb_ec2.params))

ROC_AUC Score: 0.5925454728674359
Best hyperparameters: {'n_estimators': 2047, 'max_depth': 2, 'learning_rate': 0.0354963497491049, 'subsample': 0.7412371799723116, 'reg_alpha': 0, 'reg_lambda': 0, 'colsample_bylevel': 0.6709273064478266}


In [33]:
xgb_ec2 = XGBClassifier(**trial_xgb_ec2.params)
xgb_ec2.fit(x_train, y_train['EC2'], eval_set=[(x_val, y_val['EC2'])], early_stopping_rounds=20, verbose=False)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.6709273064478266, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0354963497491049,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=2047, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

# 7. Ensemble

In [34]:
def get_scores(model,x):
    y_pred_proba = model.predict_proba(x_test)[:, 1]
    roc_auc = roc_auc_score(y_test[x], y_pred_proba)
    return roc_auc

In [35]:
ec1_scores=[get_scores(lgbm_ec1,'EC1'),get_scores(cb_ec1,'EC1'),get_scores(xgb_ec1,'EC1')]
ec2_scores=[get_scores(lgbm_ec1,'EC2'),get_scores(cb_ec1,'EC2'),get_scores(xgb_ec1,'EC2')]

In [36]:
ec1_models = [('lgbm',LGBMClassifier(**trial_lgbm_ec1.params,verbose=-1)),
             ('cat',CatBoostClassifier(**trial_cb_ec1.params,verbose=100)),
             ('xgb',XGBClassifier(**trial_xgb_ec1.params,verbose=100))]
ec2_models = [('lgbm',LGBMClassifier(**trial_lgbm_ec2.params,verbose=-1)),
             ('cat',CatBoostClassifier(**trial_cb_ec2.params,verbose=100)),
             ('xgb',XGBClassifier(**trial_xgb_ec2.params,verbose=100))]

In [37]:
ensemble_ec1 = VotingClassifier(estimators=ec1_models, voting='soft', weights=ec1_scores)
ensemble_ec1.fit(x_train_full,y_train_full['EC1'])

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] feature_fraction is set=0.8377231358054495, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8377231358054495
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0 will be ignored. Current value: lambda_l2=2
0:	learn: 0.6800288	total: 40.4ms	remaining: 2m 15s
100:	learn: 0.4768567	total: 4.44s	remaining: 2m 23s
200:	learn: 0.4095500	total: 8.68s	remaining: 2m 16s
300:	learn: 0.3472451	total: 13.2s	remaining: 2m 14s
400:	learn: 0.2956497	total: 17.5s	remaining: 2m 9s
500:	learn: 0.2523719	total: 21.8s	remaining: 2m 4s
600:	learn: 0.2174922	total: 26.2s	remaining: 2m
700:	learn: 0.1879529	total: 30.4s	remaining: 1m 55s
800:	learn: 0.1654067	total: 34.7s	remaining: 1m 50s
900:	learn: 0.1466552	total: 38.9s	remaining: 1m 46s
1000:	learn: 0.1302604	total: 43.4s	remaining: 1m 42s
1100:	learn: 0.1153569	total: 47.8s	remaining: 1m 37s
1200:	learn: 0.10324

VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(feature_fraction=0.8377231358054495,
                                             lambda_l1=0, lambda_l2=2,
                                             learning_rate=0.09383073807047086,
                                             max_depth=9, n_estimators=290,
                                             num_leaves=33, reg_alpha=0,
                                             reg_lambda=0,
                                             subsample=0.5149475034484587,
                                             verbose=-1)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x79b40d1142b0>),
                             ('xgb',
                              XGBClassifier(base_sco...
                                            learning_rate=0.0010052496318172302,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=2500, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...))],
                 voting='soft',
                 weights=[0.6948145557172694, 0.6994613915773678,
                          0.6960738620789584])

In [38]:
get_scores(ensemble_ec1,'EC1'), ec1_scores

(0.6943489492244352,
 [0.6948145557172694, 0.6994613915773678, 0.6960738620789584])

In [39]:
ensemble_ec2 = VotingClassifier(estimators=ec2_models, voting='soft', weights=ec2_scores)
ensemble_ec2.fit(x_train_full,y_train_full['EC2'])

[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.7713094644346348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7713094644346348
[LightGBM] [Warning] lambda_l2 is set=4, reg_lambda=0 will be ignored. Current value: lambda_l2=4
0:	learn: 0.6895621	total: 5.09ms	remaining: 18.1s
100:	learn: 0.5287750	total: 482ms	remaining: 16.5s
200:	learn: 0.5021233	total: 971ms	remaining: 16.2s
300:	learn: 0.4958847	total: 1.46s	remaining: 15.8s
400:	learn: 0.4929951	total: 1.94s	remaining: 15.3s
500:	learn: 0.4907710	total: 2.44s	remaining: 14.9s
600:	learn: 0.4888580	total: 2.92s	remaining: 14.4s
700:	learn: 0.4870145	total: 3.41s	remaining: 13.9s
800:	learn: 0.4852508	total: 3.89s	remaining: 13.4s
900:	learn: 0.4834801	total: 4.37s	remaining: 12.9s
1000:	learn: 0.4817145	total: 4.84s	remaining: 12.4s
1100:	learn: 0.4797935	total: 5.32s	remaining: 11.9s
1200:	learn: 0.4779041	tot

VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(feature_fraction=0.7713094644346348,
                                             lambda_l1=1, lambda_l2=4,
                                             learning_rate=0.0011403507662051577,
                                             max_depth=1, n_estimators=806,
                                             num_leaves=25, reg_alpha=0,
                                             reg_lambda=0,
                                             subsample=0.6308589121580126,
                                             verbose=-1)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x79b40d117fa0>),
                             ('xgb',
                              XGBClassifier(base_s...
                                            learning_rate=0.0354963497491049,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=2,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=2047, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...))],
                 voting='soft',
                 weights=[0.46489608426016094, 0.47359694531290036,
                          0.4603018809902107])

In [40]:
get_scores(ensemble_ec2,'EC2'), ec2_scores

(0.6160986110399261,
 [0.46489608426016094, 0.47359694531290036, 0.4603018809902107])

# 8. Preparing submission

In [41]:
x_test

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,ExactMolWt,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,HallKierAlpha,HeavyAtomMolWt,Kappa3,MaxAbsEStateIndex,MinEStateIndex,NumHeteroatoms,PEOE_VSA10,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2
14343,0.086159,0.085014,0.067604,0.063483,0.084467,0.078644,0.081599,0.057771,0.135985,0.119919,0.075567,0.996761,0.997543,0.998075,0.694737,0.079643,0.065636,0.672983,0.416767,0.261905,0.061864,0.024747,0.0,0.000000,0.000000,0.219487,0.051607,0.083090,0.162265,0.250,0.250
13008,0.311083,0.259483,0.210701,0.197856,0.261616,0.243579,0.250352,0.239694,0.232482,0.416338,0.244632,0.997095,0.997784,0.998444,0.580117,0.256555,0.069108,0.806006,0.062688,0.666667,0.371274,0.048645,0.0,0.000000,0.305717,0.856321,0.103327,0.439296,0.268910,0.000,0.000
12068,0.174533,0.191058,0.196687,0.184696,0.229396,0.213581,0.252654,0.259404,0.043017,0.066111,0.183947,0.996191,0.996230,0.996270,0.812865,0.179014,0.065730,0.718993,0.102313,0.214286,0.000000,0.032430,0.0,0.000000,0.000000,0.193770,0.108332,0.155663,0.153342,0.000,0.000
12390,0.075167,0.084583,0.062412,0.058607,0.063456,0.059081,0.052061,0.044495,0.032221,0.113202,0.071134,0.997260,0.997833,0.998282,0.727485,0.072758,0.065233,0.651453,0.374485,0.071429,0.058871,0.012373,0.0,0.114730,0.000000,0.073930,0.000000,0.000000,0.099852,0.125,0.125
12715,0.036635,0.058038,0.045627,0.042845,0.050877,0.047370,0.040615,0.000000,0.066380,0.000000,0.058594,0.997907,0.999103,0.998805,0.842105,0.060957,0.065349,0.687187,0.449440,0.071429,0.000000,0.101522,0.0,0.000000,0.000000,0.219762,0.000000,0.041545,0.071211,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6539,0.076987,0.124190,0.120919,0.139634,0.152516,0.166363,0.142034,0.051568,0.038291,0.196985,0.117662,0.997418,0.998207,0.998656,0.832749,0.119996,0.070307,0.725940,0.180433,0.190476,0.000000,0.231962,0.0,0.000000,0.000000,0.257203,0.012388,0.200044,0.101099,0.000,0.000
1471,0.272046,0.254120,0.192378,0.239023,0.267926,0.344508,0.345903,0.239694,0.279230,0.119919,0.244632,0.997010,0.998057,0.997909,0.753216,0.256555,0.068178,0.793095,0.060968,0.500000,0.375000,0.044224,0.0,0.032736,0.181180,0.290654,0.115994,0.318497,0.269562,0.000,0.000
976,0.160384,0.162239,0.125873,0.149141,0.136695,0.168741,0.173900,0.186210,0.021508,0.196985,0.252670,0.997075,0.997687,0.998444,0.901754,0.232015,0.067382,0.645599,0.119022,0.190476,0.187500,0.016215,0.0,0.000000,0.000000,0.096885,0.088520,0.097182,0.142814,0.000,0.000
14032,0.041998,0.073508,0.064604,0.060666,0.068142,0.063444,0.061732,0.045522,0.033024,0.000000,0.073367,0.997418,0.998207,0.998723,0.694737,0.075722,0.065283,0.641471,0.430196,0.095238,0.061864,0.012373,0.0,0.060717,0.000000,0.150899,0.038324,0.041545,0.106265,0.125,0.125


In [42]:
scaled_test = ms.fit_transform(test.drop(['id'],axis=1))

# 9. Submission

In [43]:
sub = pd.DataFrame({
    "id":test['id'],
    "EC1":ensemble_ec1.predict_proba(scaled_test)[:, 1],
    "EC2":ensemble_ec2.predict_proba(scaled_test)[:, 1]
})

In [44]:
sub

,id,EC1,EC2
0,14838,0.439999,0.884735
1,14839,0.862360,0.823504
2,14840,0.909445,0.830138
3,14841,0.873707,0.856673
4,14842,0.606773,0.815481
...,...,...,...
9888,24726,0.691554,0.860636
9889,24727,0.880800,0.853519
9890,24728,0.434957,0.882547
9891,24729,0.296313,0.888158


In [45]:
sub.to_csv('submission.csv',index=False)